<a href="https://colab.research.google.com/github/RishabhVenkat/SP_DOA_project/blob/main/MLP_Linear_Gaussian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


In [ ]:
class SimpleMLP(nn.Module):
  def __init__(self,input_dim):
    super(SimpleMLP,self).__init__()
    self.fc1=nn.Linear(input_dim,256)
    self.fc2=nn.Linear(256,128)
    self.fc3=nn.Linear(128,64)
    self.fc4=nn.Linear(64,input_dim)

  def forward(self,x):
    x=torch.relu(self.fc1(x))
    x=torch.relu(self.fc2(x))
    x=torch.relu(self.fc3(x))
    x=self.fc4(x)
    return x


In [ ]:
class AffineCouplingLayer(nn.Module):
  def __init__(self,input_dim):
    super(AffineCouplingLayer,self).__init__()
    self.scale=nn.Parameter(torch.ones(input_dim))
    self.shift=nn.Parameter(torch.zeros(input_dim))

  def forward(self,x):
    return x*self.scale+self.shift

  def inverse(self,y):
    return (y-self.shift)/self.scale


In [ ]:
class NormalizingFlow(nn.Module):
  def __init__(self,input_dim):
    super(NormalizingFlow,self).__init__()
    self.affine_coupling=AffineCouplingLayer(input_dim);
    self.mlp=SimpleMLP(input_dim)

  def forward(self,z,theta):
    A_theta=self.mlp(theta)
    z_transformed=self.affine_coupling(z)
    return A_theta+z_transformed

  def inverse(self,x,theta):
    A_theta=self.mlp(theta)
    z_untransformed=x-A_theta
    return self.affine_coupling.inverse(z_untransformed)


In [ ]:
def generate_synthetic_data(A,L,z):
  #return torch.matmul(z,L.T)+torch.matmul(z,A.T)
  assert z.shape[1]==A.shape[0], "Dimensions of z and A must match for multiplication."

  x=A@z.T+L@torch.randn_like(z.T);
  x=x.T
  return x


In [ ]:
def optimize_theta(flow,x,A,L,n,lr=0.00001,n_iterations=10000):
  theta=nn.Parameter(torch.randn(n))
  optimizer=optim.Adam([theta],lr=lr,weight_decay=1e-4)
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.9)
  loss_fn=nn.MSELoss()

  for _ in range(n_iterations):
    optimizer.zero_grad()
    z=flow.inverse(x,theta)
    x_reconstructed=flow(z,theta)
    loss=loss_fn(x,x_reconstructed)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(theta,max_norm=1.0)
    optimizer.step()
    scheduler.step()

  return theta


In [ ]:
np.random.seed(42)
torch.manual_seed(42)
n=5
A=torch.randn(n,n)
C_vv=np.cov(np.random.randn(n,100))
L=torch.tensor(np.linalg.cholesky(C_vv).astype(np.float32))

true_theta=2.0


In [ ]:
n_samples=1000
z=torch.randn(n_samples,n)
x=generate_synthetic_data(A,L,z)

x_mean=x.mean(dim=0)
x_std=x.std(dim=0)
x=(x-x_mean)/x_std

flow=NormalizingFlow(n)

estimated_theta=optimize_theta(flow,x,A,L,n)
difference=estimated_theta-true_theta
print("Difference between estimated theta and true theta:",difference)


Difference between estimated theta and true theta: tensor([-2.9069,  0.2775, -0.6193, -0.2486, -2.9741], grad_fn=<SubBackward0>)


In [ ]:
def fisher_information_matrix(x,theta,flow):
  x=x.clone().detach().requires_grad_(True)
  z=flow.inverse(x,theta)
  reconstructed_x=flow(z,theta)
  loss_fn=nn.MSELoss()
  loss=loss_fn(x,reconstructed_x)
  loss.backward(retain_graph=True)
  grad=torch.autograd.grad(loss,theta,create_graph=True)[0]

  if grad.dim()==1:
    grad=grad.unsqueeze(0)

  FIM=torch.matmul(grad.T,grad)
  return FIM


In [ ]:
def add_regularization(matrix,epsilon=1e-6):
  return matrix+torch.eye(matrix.size(0),device=matrix.device)*epsilon


In [ ]:
FIM_estimated=fisher_information_matrix(x,estimated_theta,flow)
if FIM_estimated.ndimension()==1:
  FIM_estimated=FIM_estimated.unsqueeze(0).unsqueeze(0)

print("Shape of estimated FIM:",FIM_estimated.shape)
FIM_estimated=add_regularization(FIM_estimated)

try:
  CRB_estimated=torch.inverse(FIM_estimated)
  print("CRB Estimation Successful")
except RuntimeError as e:
  print("Error computing inverse:",e)


Shape of estimated FIM: torch.Size([5, 5])
CRB Estimation Successful


In [ ]:
epsilon_values=[1,2,10]
for epsilon in epsilon_values:
  FIM_estimated=fisher_information_matrix(x,estimated_theta,flow)
  FIM_estimated=add_regularization(FIM_estimated,epsilon)
  try:
    CRB_estimated=torch.inverse(FIM_estimated)
    print(f"Estimated CRB with epsilon={epsilon}:{torch.det(FIM_estimated)}")
    print
  except RuntimeError as e:
    print(f"Error with epsilon={epsilon}:{e}")


Estimated CRB with epsilon=1:1.0
Estimated CRB with epsilon=2:32.0
Estimated CRB with epsilon=10:100000.0


In [ ]:
def actual_fisher_information_matrix(A,C_vv):
  return torch.matmul(A.T,torch.inverse(torch.tensor(C_vv,dtype=torch.float32)))@A


In [ ]:
FIM_actual=actual_fisher_information_matrix(A,C_vv)
CRB_actual=torch.inverse(FIM_actual)


In [ ]:
print("True theta:",true_theta)
print("Estimated theta:",estimated_theta)
print("Actual Fisher Information Matrix:",FIM_actual)
print("Estimated Fisher Information Matric:",FIM_estimated)
print("Actual CRB:",CRB_actual)
print("Estimated CRB:",CRB_estimated)


True theta: 2.0
Estimated theta: Parameter containing:
tensor([ 0.8803, -1.1164,  0.4475,  0.0417, -1.2856], requires_grad=True)
Actual Fisher Information Matrix: tensor([[ 6.3663,  3.1568,  4.8521, -4.2329,  4.4982],
        [ 3.1568,  4.5754,  1.6776, -4.5801,  1.4389],
        [ 4.8521,  1.6776,  5.2751, -1.4182,  5.4028],
        [-4.2329, -4.5801, -1.4182,  7.1255, -2.7055],
        [ 4.4982,  1.4389,  5.4028, -2.7055,  8.6176]])
Estimated Fisher Information Matric: tensor([[10.,  0.,  0.,  0.,  0.],
        [ 0., 10.,  0.,  0.,  0.],
        [ 0.,  0., 10.,  0.,  0.],
        [ 0.,  0.,  0., 10.,  0.],
        [ 0.,  0.,  0.,  0., 10.]], grad_fn=<AddBackward0>)
Actual CRB: tensor([[ 1798.3230,   878.3101, -2516.9656,  1497.4629,   962.8124],
        [  878.3102,   429.8562, -1229.7511,   731.9708,   470.5673],
        [-2516.9658, -1229.7512,  3523.5557, -2096.2080, -1348.0776],
        [ 1497.4631,   731.9708, -2096.2080,  1247.5072,   802.0198],
        [  962.8125,   470.5674,

In [ ]:
difference=torch.norm(CRB_actual-CRB_estimated).item()
print("Difference between actual and estimated CRB:",difference)



Difference between actual and estimated CRB: 7513.67724609375
